# CHAPTER 11 Time Series（时间序列）

时间序列指能在任何能在时间上观测到的数据。很多时间序列是有固定频率（fixed frequency）的，意思是数据点会遵照某种规律定期出现，比如每15秒，每5分钟，或每个月。时间序列也可能是不规律的（irregular），没有一个固定的时间规律。如何参照时间序列数据取决于我们要做什么样的应用，我们可能会遇到下面这些：

- Timestamps（时间戳），具体的某一个时刻
- Fixed periods（固定的时期），比如2007年的一月，或者2010年整整一年
- Intervals of time（时间间隔），通常有一个开始和结束的时间戳。Periods（时期）可能被看做是Intervals（间隔）的一种特殊形式。
- Experiment or elapsed time（实验或经过的时间）；每一个时间戳都是看做是一个特定的开始时间（例如，在放入烤箱后，曲奇饼的直径在每一秒的变化程度）

这一章主要涉及前三个类型。

> pandas也支持基于timedeltas的index，本书不会对timedelta index做介绍，感兴趣的可以查看pandas的文档。


# 11.1 Date and Time Data Types and Tools（日期和时间数据类型及其工具）

python有标准包用来表示时间和日期数据。datetime, time, calendar，这些模块经常被使用。datetime.datetime类型，或简单写为datetime，被广泛使用：


In [27]:
import pandas as pd

In [1]:
from datetime import datetime

In [3]:
now = datetime.now()

In [4]:
now

datetime.datetime(2017, 12, 1, 12, 12, 0, 375896)

In [5]:
now.year, now.month, now.day

(2017, 12, 1)

datetime能保存日期和时间到微妙级别。timedelta表示两个不同的datetime对象之间的时间上的不同：

In [6]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(926, 56700)

In [7]:
delta.days

926

In [8]:
delta.seconds

56700

我们可以在一个datetime对象上，添加或减少一个或多个timedelta，这样可以产生新的变化后的对象：

In [9]:
from datetime import timedelta

In [10]:
start = datetime(2011, 1, 7)

In [11]:
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [12]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

下表汇总了一些datetime模块中的数据类型：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/wqo6m.png)

# 1 Converting Between String and Datetime（字符串与时间的转换）

我们可以对datetime对象，以及pandas的Timestamp对象进行格式化，这部分之后会介绍，使用str或strftime方法，传入一个特定的时间格式就能进行转换：

In [13]:
stamp = datetime(2011, 1, 3)

In [14]:
str(stamp)

'2011-01-03 00:00:00'

In [15]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

下表是关于日期时间类型的格式：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/r98dw.png)

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/bc9e8.png)

我们可以利用上面的format codes（格式码；时间日期格式）把字符串转换为日期，这要用到datetime.strptime:

In [16]:
value = '2011-01-03'

In [17]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [18]:
datestrs = ['7/6/2011', '8/6/2011']

In [19]:
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

对于一个一直的时间格式，使用datetime.strptime来解析日期是很好的方法。但是，如果每次都要写格式的话很烦人，尤其是对于一些比较常见的格式。在这种情况下，我们可以使用第三方库dateutil中的parser.parse方法（这个库会在安装pandas的时候自动安装）：

In [20]:
from dateutil.parser import parse

In [21]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil能够解析很多常见的时间表示格式：

In [22]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

在国际上，日在月之前是很常见的（译者：美国是把月放在日前面的），所以我们可以设置dayfirst=True来指明最前面的是否是日：

In [23]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

pandas通常可以用于处理由日期组成的数组，不论是否是DataFrame中的行索引或列。to_datetime方法能解析很多不同种类的日期表示。标准的日期格式，比如ISO 8601，能被快速解析：

In [24]:
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

In [28]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

还能处理一些应该被判断为缺失的值（比如None, 空字符串之类的）：

In [29]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [30]:
idx[2]

NaT

In [31]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

Nat(Not a Time)在pandas中，用于表示时间戳为空值（null value）。

> dateutil.parse是一个很有用但不完美的工具。它可能会把一些字符串识别为日期，例如，'42'就会被解析为2042年加上今天的日期。

datetime对象还有一些关于地区格式（locale-specific formatting）的选项，用于处理不同国家或不同语言的问题。例如，月份的缩写在德国和法国，与英语是不同的。下表列出一些相关的选项：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/gp2fy.png)